In [1]:
import numpy as np
from scipy.optimize import minimize
from tqdm import tqdm
from scipy.stats import t, multivariate_normal, multivariate_t


import plotly.graph_objects as go
from plotly.subplots import make_subplots


# from simulation import *

# Univariate Linear State Space Model

In [12]:
# def simulate_univariate_t_gas(T, phi, kappa, sigma, nu, beta=None, seed=None):
#     """
#     Simulate univariate GAS model data explicitly, optionally including regressors.

#     Model explicitly stated:
#         y_t = mu_t + X_t @ beta + eps_t, eps_t ~ t_nu(0, sigma^2)
#         mu_{t+1} = phi * mu_t + kappa * u_t,
#         with:
#         u_t = ((nu + 1) * (y_t - mu_t - X_t @ beta)) / (nu * sigma^2 + (y_t - mu_t - X_t @ beta)^2)

#     Parameters:
#     -----------
#     T : int
#         Number of observations explicitly.
#     phi : float
#         AR(1) parameter explicitly.
#     kappa : float
#         GAS updating parameter explicitly.
#     sigma : float
#         Scale parameter explicitly of the Student's t-distribution.
#     nu : float
#         Degrees of freedom explicitly for the Student's t-distribution.
#     beta : ndarray (K,) or None, optional
#         Explicit coefficient vector for regressors. If None, explicitly no regressors included.
#     seed : int or None, optional
#         Explicit random seed for reproducibility.

#     Returns:
#     --------
#     y : ndarray (T,)
#         Simulated observations explicitly.
#     mu : ndarray (T,)
#         Simulated latent states explicitly.
#     X : ndarray (T, K) or None
#         Simulated regressors explicitly if provided; otherwise None explicitly.
#     """

#     if seed is not None:
#         np.random.seed(seed)

#     # Explicitly handle optional regressors
#     if beta is not None:
#         beta = np.asarray(beta)
#         K = len(beta)
#         X = np.random.randn(T, K)
#     else:
#         K = 0
#         X = None

#     mu = np.zeros(T)
#     y = np.zeros(T)

#     for i in range(1, T):
#         # Explicitly set prediction with or without regressors
#         if beta is not None:
#             pred = mu[i-1] + X[i] @ beta
#         else:
#             pred = mu[i-1]

#         # Explicitly generate Student's t noise
#         eps_i = t.rvs(df=nu) * sigma
#         y[i] = pred + eps_i

#         # Compute residual explicitly
#         v_i = y[i] - pred

#         # Explicit GAS scaled score update (correct form)
#         u_i =  1 / (1+ (v_i**2)/ (nu + sigma** 2)) # ((nu + 1) * v_i) / (nu * sigma**2 + v_i**2)

#         # Explicit GAS latent state update
#         mu[i] = phi * mu[i-1] + kappa * u_i

#     return y, mu, X

import numpy as np

def simulate_univariate_state_space(T, K, c, phi, beta, Q, R, seed=None, use_intercept=True):
    """
    Simulate a univariate linear Gaussian state-space model with optional intercept in regressors.

    Model:
        y_t = mu_t + X_t @ beta + epsilon_t, epsilon_t ~ N(0, R)
        mu_{t+1} = c + phi * mu_t + eta_t, eta_t ~ N(0, Q)

    Parameters:
    -----------
    T : int
        Number of time points.
    K : int
        Number of regressors (including intercept if use_intercept=True).
    c : float
        State intercept.
    phi : float
        State AR(1) parameter.
    beta : ndarray (K,)
        Regressor coefficients. First element is intercept if use_intercept is True.
    Q : float
        State noise variance.
    R : float
        Observation noise variance.
    seed : int or None
        Random seed.
    use_intercept : bool
        If True, treat the first element of beta as intercept and prepend a column of ones to X.

    Returns:
    --------
    y : ndarray (T,)
        Simulated observations.
    mu : ndarray (T,)
        Latent states.
    X : ndarray (T, K)
        Simulated regressors (with intercept if used).
    """
    if seed is not None:
        np.random.seed(seed)

    beta = np.asarray(beta)
    assert beta.shape == (K,), f"Expected beta of shape ({K},), got {beta.shape}"

    if use_intercept:
        X_raw = np.random.normal(size=(T, K - 1))
        X = np.ones((T, K))
        X[:, 1:] = X_raw
    else:
        X = np.random.normal(size=(T, K))

    mu = np.zeros(T)
    y = np.zeros(T)

    mu[0] = c / (1 - phi)

    for t in range(T):
        y[t] = mu[t] + X[t] @ beta + np.random.normal(scale=np.sqrt(R))
        if t < T - 1:
            mu[t + 1] = c + phi * mu[t] + np.random.normal(scale=np.sqrt(Q))

    return y, mu, X




In [18]:
T = 300
phi = 0.7
c = 0.2
Q = 0.1
R = 0.3
beta = np.array([1.0, 0.5, -0.3])  # intercept + 2 regressors
K = len(beta)

y_uss, mu_uss, X_uss = simulate_univariate_state_space(T, K, c, phi, beta, Q, R, seed=123, use_intercept=True)


In [20]:
time_axis = np.arange(T)

fig = go.Figure()

# Explicitly plot simulated observations
fig.add_trace(go.Scatter(
    x=time_axis, y=y_uss, mode='lines',
    name='Simulated Observations (y)', line=dict(width=1.5)
))

# Explicitly plot latent state
fig.add_trace(go.Scatter(
    x=time_axis, y=mu_uss, mode='lines',
    name='Latent State (μ)', line=dict(width=2, dash='dash')
))

for i in range(X_uss.shape[1]):
    fig.add_trace(go.Scatter(
        x=time_axis, y=X_uss[:, i], mode='lines',
        name=f'Regressor {i+1}', line=dict(width=1.5, dash='dot')
    ))

fig.update_layout(
    title='Explicit GAS Univariate Simulation',
    xaxis_title='Time',
    yaxis_title='Value',
    legend=dict(x=1.05, y=1),
    template='plotly_white',
    width=950,
    height=550
)

fig.show()

# Univariate GAS Model

In [21]:
# def simulate_gas_data_univariate(T, phi, kappa, sigma, nu, beta=None, seed=None):
#     """
#     Simulate univariate GAS model data explicitly, optionally with regressors.

#     Model explicitly:
#         y_t = mu_t + X_t @ beta + eps_t, eps_t ~ t_nu(0, sigma^2)

#         mu_{t+1} = phi * mu_t + kappa * u_t,

#         Explicitly scaled GAS score:
#         u_t = ((nu + 1) * v_t) / (nu * sigma^2 + v_t^2),
#         where v_t = y_t - mu_t - X_t @ beta

#     Parameters:
#     -----------
#     T : int
#         Number of observations explicitly.
#     phi : float
#         AR(1) parameter explicitly for latent state.
#     kappa : float
#         GAS updating parameter explicitly.
#     sigma : float
#         Scale parameter explicitly of Student's t-distribution.
#     nu : float
#         Degrees of freedom explicitly for Student's t-distribution.
#     beta : ndarray (K,) or None, optional
#         Coefficient vector explicitly for regressors. If None, explicitly no regressors.
#     seed : int or None, optional
#         Random seed explicitly for reproducibility.

#     Returns:
#     --------
#     y : ndarray (T,)
#         Simulated observations explicitly.
#     mu : ndarray (T,)
#         Simulated latent states explicitly.
#     X : ndarray (T, K) or None
#         Simulated regressors explicitly if provided; otherwise None explicitly.
#     """

#     if seed is not None:
#         np.random.seed(seed)

#     if beta is not None:
#         beta = np.asarray(beta)
#         K = beta.shape[0]
#         X = np.random.randn(T, K)
#     else:
#         K = 0
#         X = None

#     mu = np.zeros(T)
#     y = np.zeros(T)

#     for i in range(1, T):
#         if beta is not None:
#             pred = mu[i-1] + X[i] @ beta
#         else:
#             pred = mu[i-1]

#         # Generate explicitly Student's t noise
#         eps_i = t.rvs(df=nu) * sigma
#         y[i] = pred + eps_i

#         # Explicitly compute residual
#         v_i = y[i] - pred

#         # Explicitly scaled GAS score (correct definition)
#         # u_i = ((nu + 1) * v_i) / (nu * sigma**2 + v_i**2)
#         weight = 1 / (1 + (v_i**2) / (nu * sigma**2))
#         u_i = weight * v_i

#         # Explicit GAS latent state update
#         mu[i] = phi * mu[i-1] + kappa * u_i

#     return y, mu, X
def simulate_univariate_t_gas(T, phi, kappa, sigma, nu, beta=None, seed=None, use_intercept=True):
    """
    Simulate univariate GAS model data with optional regressors and intercept.

    Model:
        y_t = mu_t + X_t @ beta + eps_t, eps_t ~ t_nu(0, sigma^2)
        mu_{t+1} = phi * mu_t + kappa * u_t
        u_t = scaled score

    Parameters:
    -----------
    T : int
        Number of observations.
    phi : float
        AR(1) parameter.
    kappa : float
        GAS updating parameter.
    sigma : float
        Scale parameter of the Student's t-distribution.
    nu : float
        Degrees of freedom.
    beta : ndarray (K,) or None
        Regression coefficients. If None, no regressors.
    seed : int or None
        Random seed for reproducibility.
    use_intercept : bool
        If True, include intercept column (first beta element is intercept).

    Returns:
    --------
    y : ndarray (T,)
        Simulated observations.
    mu : ndarray (T,)
        Simulated latent states.
    X : ndarray (T, K) or None
        Simulated regressors (with intercept if used), or None if beta is None.
    """
    if seed is not None:
        np.random.seed(seed)

    if beta is not None:
        beta = np.asarray(beta)
        K = len(beta)
        X = np.random.randn(T, K - 1) if use_intercept else np.random.randn(T, K)
        if use_intercept:
            X = np.hstack((np.ones((T, 1)), X))  # Add intercept column
    else:
        K = 0
        X = None

    mu = np.zeros(T)
    y = np.zeros(T)

    for i in range(1, T):
        # Prediction
        pred = mu[i - 1]
        if beta is not None:
            pred += X[i] @ beta

        eps_i = t.rvs(df=nu) * sigma
        y[i] = pred + eps_i

        v_i = y[i] - pred
        u_i = 1 / (1 + (v_i ** 2) / (nu * sigma ** 2))

        mu[i] = phi * mu[i - 1] + kappa * u_i

    return y, mu, X



In [22]:
# # Explicit parameters for GAS simulation without regressors
# T = 500
# phi_true = 0.8
# kappa_true = 0.7
# sigma_true = 0.8
# nu_true = 5

# # Run explicit GAS simulation without regressors
# y_sim, mu_sim, X_sim = simulate_univariate_t_gas(
#     T=T,
#     phi=phi_true,
#     kappa=kappa_true,
#     sigma=sigma_true,
#     nu=nu_true,
#     beta=None,  # Explicitly no regressors
#     seed=8888
# )

# print("✅ GAS simulation without regressors completed explicitly.")

# Explicit parameters for GAS simulation with regressors
T = 500
phi_true = 0.8
kappa_true = 0.7
beta_true = np.array([2.0, -1.5])
sigma_true = 0.8
nu_true = 10.0

# Run explicit GAS simulation with regressors
y_sim, mu_sim, X_sim = simulate_univariate_t_gas(
    T=T,
    phi=phi_true,
    kappa=kappa_true,
    sigma=sigma_true,
    nu=nu_true,
    beta=beta_true,
    use_intercept=True,  # Explicit regressors provided
    seed=8888
)

# # Simulate with 2 regressors and intercept
# T = 300
# phi = 0.8
# kappa = 0.6
# sigma = 1.0
# nu = 10
# beta = np.array([1.0, 0.5, -0.3])  # Intercept + 2 regressors

# y, mu, X = simulate_univariate_t_gas(T, phi, kappa, sigma, nu, beta, seed=42, use_intercept=True)

print("✅ GAS simulation with regressors completed explicitly.")

✅ GAS simulation with regressors completed explicitly.


In [23]:
time_axis = np.arange(T)

fig = go.Figure()

# Explicitly plot simulated observations
fig.add_trace(go.Scatter(
    x=time_axis, y=y_sim, mode='lines',
    name='Simulated Observations (y)', line=dict(width=1.5)
))

# Explicitly plot latent state
fig.add_trace(go.Scatter(
    x=time_axis, y=mu_sim, mode='lines',
    name='Latent State (μ)', line=dict(width=2, dash='dash')
))

for i in range(X_sim.shape[1]):
    fig.add_trace(go.Scatter(
        x=time_axis, y=X_sim[:, i], mode='lines',
        name=f'Regressor {i+1}', line=dict(width=1.5, dash='dot')
    ))

fig.update_layout(
    title='Explicit GAS Univariate Simulation',
    xaxis_title='Time',
    yaxis_title='Value',
    legend=dict(x=1.05, y=1),
    template='plotly_white',
    width=950,
    height=550
)

fig.show()

# Multivariate Linear State Space Model

In [11]:
def simulate_multivariate_state_space(T, N, c, Phi, Q, R, beta = None, seed = None):
    """
    Simulate a multivariate linear Gaussian state-space model explicitly, optionally with regressors.

    Observation equation:
        y_t = mu_t + X_t @ beta + epsilon_t, epsilon_t ~ N(0, R)

    State equation:
        mu_{t+1} = c + Phi @ mu_t + eta_t, eta_t ~ N(0, Q)

    Parameters:
    -----------
    T : int
        Number of observations explicitly.
    N : int
        Dimension of the response vector explicitly.
    c : ndarray (N,)
        State intercept explicitly.
    Phi : ndarray (N, N)
        State transition matrix explicitly.
    Q : ndarray (N, N)
        State noise covariance explicitly.
    R : ndarray (N, N)
        Observation noise covariance explicitly.
    beta : ndarray (K, N) or None, optional
        Regression coefficient matrix explicitly. If None, explicitly no regressors.
    seed : int or None, optional
        Random seed explicitly for reproducibility.

    Returns:
    --------
    y : ndarray (T, N)
        Simulated multivariate observations explicitly.
    mu : ndarray (T, N)
        Simulated latent states explicitly.
    X : ndarray (T, K) or None
        Simulated regressors explicitly if provided; otherwise, None explicitly.
    """    """
    Simulate a multivariate linear Gaussian state-space model explicitly, optionally with regressors.

    Parameters:
    -----------
    T : int
        Number of observations explicitly.
    N : int
        Dimension of the response vector explicitly.
    c : ndarray (N,)
        State intercept explicitly.
    Phi : ndarray (N, N)
        State transition matrix explicitly.
    Q : ndarray (N, N)
        State noise covariance explicitly.
    R : ndarray (N, N)
        Observation noise covariance explicitly.
    beta : ndarray (K, N) or None, optional
        Regression coefficient matrix explicitly. If None, explicitly no regressors.
    seed : int or None, optional
        Random seed explicitly for reproducibility.

    Returns:
    --------
    y : ndarray (T, N)
        Simulated multivariate observations explicitly.
    mu : ndarray (T, N)
        Simulated latent states explicitly.
    X : ndarray (T, K) or None
        Simulated regressors explicitly if provided; otherwise, None explicitly.
    """

    if seed is not None:
        np.random.seed(seed)
    
    # Validate dimensions
    c = np.asarray(c).reshape(N, )
    Phi = np.asarray(Phi).reshape(N, N)
    Q = np.asarray(Q).reshape(N, N)
    R = np.asarray(R).reshape(N, N)

    # Check if regressors
    if beta is not None:
        beta = np.asarray(beta)
        K = beta.shape[0]
        X = np.random.normal(size=(T, K))
    else:
        K = 0
        X = None
    
    mu = np.zeros((T, N))
    y = np.zeros((T, N))

    # Initial state
    mu[0] = np.linalg.solve(np.eye(N) - Phi, c)

    for i in range(T):
        # Observation Equation
        if beta is not None:
            y[i] = mu[i] + X[i] @ beta + np.random.multivariate_normal(mean = np.zeros(N), cov = R)
        else:
            y[i] = mu[i] + np.random.multivariate_normal(mean = np.zeros(N), cov = R)
        
        # State Equation
        if i < T - 1:
            mu[i + 1] = c + Phi @ mu[i] + np.random.multivariate_normal(mean = np.zeros(N), cov = Q)

    return y, mu, X

In [24]:
def simulate_multivariate_state_space(T, N, c, Phi, Q, R, beta=None, seed=None, use_intercept=True):
    """
    Simulate a multivariate linear Gaussian state-space model with optional intercept.

    Observation equation:
        y_t = mu_t + X_t @ beta + epsilon_t, epsilon_t ~ N(0, R)

    State equation:
        mu_{t+1} = c + Phi @ mu_t + eta_t, eta_t ~ N(0, Q)

    Parameters:
    -----------
    T : int
        Number of observations.
    N : int
        Dimension of the response vector.
    c : ndarray (N,)
        State intercept.
    Phi : ndarray (N, N)
        State transition matrix.
    Q : ndarray (N, N)
        State noise covariance.
    R : ndarray (N, N)
        Observation noise covariance.
    beta : ndarray (K, N) or None
        Regression coefficient matrix. If use_intercept=True, first row is intercept.
    seed : int or None
        Random seed for reproducibility.
    use_intercept : bool
        Whether to include a column of 1s for intercept in regressors.

    Returns:
    --------
    y : ndarray (T, N)
        Simulated multivariate observations.
    mu : ndarray (T, N)
        Simulated latent states.
    X : ndarray (T, K)
        Simulated regressors, with intercept column if specified.
    """
    if seed is not None:
        np.random.seed(seed)
    
    Phi = np.asarray(Phi).reshape(N, N)
    Q = np.asarray(Q).reshape(N, N)
    R = np.asarray(R).reshape(N, N)

    if beta is not None:
        beta = np.asarray(beta)
        K = beta.shape[0]
        if use_intercept:
            K_eff = K - 1
            X_raw = np.random.normal(size=(T, K_eff))
            X = np.ones((T, K))
            X[:, 1:] = X_raw
        else:
            X = np.random.normal(size=(T, K))
    else:
        X = None

    mu = np.zeros((T, N))
    y = np.zeros((T, N))

    mu[0] = np.linalg.solve(np.eye(N) - Phi, c)

    for t in range(T):
        if beta is not None:
            y[t] = mu[t] + X[t] @ beta + np.random.multivariate_normal(np.zeros(N), R)
        else:
            y[t] = mu[t] + np.random.multivariate_normal(np.zeros(N), R)

        if t < T - 1:
            mu[t + 1] = c + Phi @ mu[t] + np.random.multivariate_normal(np.zeros(N), Q)

    return y, mu, X

In [25]:
# Explicit parameters without regressors
T = 500
N = 2
c_true = np.array([0.2, -0.1])
Phi_true = np.array([[0.8, 0.1],
                     [0.05, 0.85]])
Q_true = np.array([[1.0, 0.2],
                   [0.2, 0.8]])
R_true = np.array([[0.5, 0.1],
                   [0.1, 0.3]])

# Run explicit simulation without regressors
y_sim, mu_sim, _ = simulate_multivariate_state_space(
    T=T,
    N=N,
    c=c_true,
    Phi=Phi_true,
    Q=Q_true,
    R=R_true,
    beta=None,  # Explicitly no regressors
    seed=8888
)

print("✅ Multivariate State-Space Simulation without regressors explicitly completed.")


✅ Multivariate State-Space Simulation without regressors explicitly completed.


In [26]:
time_axis = np.arange(T)
# Explicitly create subplots
fig = make_subplots(rows=N, cols=1, shared_xaxes=True,
                    subplot_titles=[f"Dimension {i+1}" for i in range(N)])

for dim in range(N):
    # Plot observations explicitly
    fig.add_trace(go.Scatter(
        x=time_axis, y=y_sim[:, dim],
        mode='lines', name=f'Observation y[{dim+1}]',
        line=dict(width=1.5), opacity=0.7
    ), row=dim+1, col=1)

    # Plot latent states explicitly
    fig.add_trace(go.Scatter(
        x=time_axis, y=mu_sim[:, dim],
        mode='lines', name=f'Latent state μ[{dim+1}]',
        line=dict(width=2, dash='dash')
    ), row=dim+1, col=1)

fig.update_layout(
    height=300*N, width=950,
    title_text="Explicit Multivariate GAS Data: Observations vs. Latent States",
    template="plotly_white"
)

fig.update_xaxes(title_text="Time", row=N, col=1)
fig.update_yaxes(title_text="Value")

fig.show()

In [32]:
# Explicit parameters with regressors
T = 500
N = 2
K = 3
c_true = np.array([0.2, -0.1])
Phi_true = np.array([[0.8, 0.1],
                     [0.05, 0.85]])
beta_true = np.array([[1.0, -0.5],
                      [-1.2, 0.8],
                      [0.5, 1.5]])
Q_true = np.array([[1.0, 0.2],
                   [0.2, 0.8]])
R_true = np.array([[0.5, 0.1],
                   [0.1, 0.3]])

# Run explicit simulation with regressors
y_sim, mu_sim, X_sim = simulate_multivariate_state_space(
    T=T,
    N=N,
    c=c_true,
    Phi=Phi_true,
    beta=beta_true,  # Explicitly include regressors
    Q=Q_true,
    R=R_true,
    # use_intercept=None,
    seed=8888
)

print("✅ Multivariate State-Space Simulation with regressors explicitly completed.")


✅ Multivariate State-Space Simulation with regressors explicitly completed.


In [33]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

time_axis = np.arange(T)
K = X_sim.shape[1]

# Create subplots: N rows for dimensions + 1 for regressors
fig = make_subplots(
    rows=N + 1, cols=1, shared_xaxes=True,
    subplot_titles=[f"Dimension {i+1}" for i in range(N)] + ["Regressors X_t"]
)

# --- Add y and mu plots per dimension ---
for dim in range(N):
    fig.add_trace(go.Scatter(
        x=time_axis, y=y_sim[:, dim],
        mode='lines', name=f'Observation y[{dim+1}]',
        line=dict(width=1.5), opacity=0.7
    ), row=dim+1, col=1)

    fig.add_trace(go.Scatter(
        x=time_axis, y=mu_sim[:, dim],
        mode='lines', name=f'Latent state μ[{dim+1}]',
        line=dict(width=2, dash='dash')
    ), row=dim+1, col=1)

# --- Plot each regressor in final subplot row ---
for k in range(K):
    fig.add_trace(go.Scatter(
        x=time_axis, y=X_sim[:, k],
        mode='lines', name=f'Regressor X[{k+1}]',
        line=dict(width=1)
    ), row=N+1, col=1)

# --- Layout styling ---
fig.update_layout(
    height=300 * (N + 1),
    width=950,
    title_text="Multivariate State-Space Simulation: Observations, Latent States, and Regressors",
    template="plotly_white",
    showlegend=True
)

fig.update_xaxes(title_text="Time", row=N+1, col=1)
fig.update_yaxes(title_text="Value")

fig.show()


# Multivariate GAS Model

In [34]:
# def simulate_multivariate_t_gas(T, N, omega, Phi, K_mat, Omega, nu, beta = None, seed = None):
#     """
#     Simulate multivariate GAS model data explicitly, optionally with regressors.

#     Model explicitly stated:
#         y_t = mu_t + X_t @ beta + eps_t, eps_t ~ t_nu(0, Omega)
#         mu_{t+1} = omega + Phi(mu_t - omega) + K_mat * u_t

#     Explicitly defined scaled score:
#         u_t = ((nu + N) * inv_Omega @ residual) / (nu + residual' @ inv_Omega @ residual)

#     Parameters:
#     -----------
#     T : int
#         Number of observations.
#     N : int
#         Dimension of the response vector.
#     omega : ndarray (N,)
#         GAS intercept explicitly.
#     Phi : ndarray (N, N)
#         GAS transition matrix explicitly.
#     K_mat : ndarray (N, N)
#         GAS updating matrix explicitly.
#     Omega : ndarray (N, N)
#         Covariance matrix (positive definite).
#     nu : float
#         Degrees of freedom for Student's t-distribution.
#     beta : ndarray (K, N) or None, optional
#         Regression coefficient matrix. If None, no regressors included explicitly.
#     seed : int or None, optional
#         Random seed for reproducibility.

#     Returns:
#     --------
#     y : ndarray (T, N)
#         Simulated observations explicitly.
#     mu : ndarray (T, N)
#         Simulated latent states explicitly.
#     X : ndarray (T, K) or None
#         Simulated regressors explicitly if beta provided, else None.
#     """
#     if seed is not None:
#         np.random.seed(seed)
    
#     # Validate dimensions
#     omega = np.asarray(omega).reshape(N, )
#     Phi = np.asarray(Phi).reshape(N, N)
#     K_mat = np.asarray(K_mat).reshape(N, N)
#     Omega = np.asarray(Omega).reshape(N, N)
#     inv_Omega = np.linalg.inv(Omega)

#     # Check if regressors explicitly provided
#     if beta is not None:
#         beta = np.asarray(beta)
#         K = beta.shape[0]
#         X = np.random.normal(size=(T, K))
#     else:
#         K = 0
#         X = None

#     mu = np.zeros((T, N))
#     y = np.zeros((T, N))
#     mu[0] = omega # Initial state

#     for i in tqdm(range(T)):
#         if beta is not None:
#             pred = mu[i-1] + X[i] @ beta
#         else:
#             pred = mu[i-1]
        
#         # Generate multivariate t-distributed noise
#         eps_t = multivariate_t.rvs(df=nu, loc=np.zeros(N), shape=Omega)
#         y[i] = pred + eps_t

#         # Compute residual
#         resid = y[i] - pred

#         # Compute mahalanobis distance
#         mahalanobis = resid @ inv_Omega @ resid 

#         # Compute scaled score
#         weight = (nu + N) / (nu + mahalanobis)
#         u_t = weight * (inv_Omega @ resid)

#         # GAS latent state update
#         mu[i] = omega + Phi @ (mu[i-1] - omega) + K_mat @ u_t

#     return y, mu, X 


def simulate_multivariate_t_gas(T, N, omega, Phi, K_mat, Omega, nu, beta=None, intercept=True, seed=None):
    """
    Simulate multivariate GAS model data, optionally with regressors and intercept.

    Model:
        y_t = mu_t + X_t @ beta + eps_t, eps_t ~ t_nu(0, Omega)
        mu_{t+1} = omega + Phi(mu_t - omega) + K_mat * u_t

    Scaled score:
        u_t = ((nu + N) * inv_Omega @ residual) / (nu + residual.T @ inv_Omega @ residual)

    Parameters:
    -----------
    T : int
        Number of time steps.
    N : int
        Dimension of the response vector.
    omega : ndarray (N,)
        GAS intercept.
    Phi : ndarray (N, N)
        Transition matrix.
    K_mat : ndarray (N, N)
        GAS score update matrix.
    Omega : ndarray (N, N)
        Covariance matrix.
    nu : float
        Degrees of freedom of Student's t.
    beta : ndarray (K or K+1, N), optional
        Regression coefficient matrix.
    intercept : bool
        If True, adds intercept column to X (first column of ones).
    seed : int, optional
        Random seed.

    Returns:
    --------
    y : ndarray (T, N)
        Simulated observations.
    mu : ndarray (T, N)
        Simulated latent state.
    X : ndarray (T, K or K+1) or None
        Regressor matrix.
    """
    if seed is not None:
        np.random.seed(seed)

    omega = np.asarray(omega).reshape(N)
    Phi = np.asarray(Phi).reshape(N, N)
    K_mat = np.asarray(K_mat).reshape(N, N)
    Omega = np.asarray(Omega).reshape(N, N)
    inv_Omega = np.linalg.inv(Omega)

    if beta is not None:
        beta = np.asarray(beta)
        K_full = beta.shape[0]
        K = K_full - 1 if intercept else K_full
        X_raw = np.random.normal(size=(T, K))
        X = np.ones((T, K_full)) if intercept else np.empty((T, K_full))
        if intercept:
            X[:, 1:] = X_raw
        else:
            X[:, :] = X_raw
    else:
        X = None

    mu = np.zeros((T, N))
    y = np.zeros((T, N))
    mu[0] = omega.copy()

    for t in tqdm(range(T)):
        if beta is not None:
            pred = mu[t - 1] + X[t] @ beta
        else:
            pred = mu[t - 1]

        eps_t = multivariate_t.rvs(df=nu, loc=np.zeros(N), shape=Omega)
        y[t] = pred + eps_t

        resid = y[t] - pred
        mahalanobis = resid @ inv_Omega @ resid
        weight = (nu + N) / (nu + mahalanobis)
        u_t = weight * (inv_Omega @ resid)

        mu[t] = omega + Phi @ (mu[t - 1] - omega) + K_mat @ u_t

    return y, mu, X


In [8]:
# Explicit simulation parameters without regressors
T = 500
N = 2
omega_true = np.array([0.5, -0.2])
Phi_true = np.array([[0.8, 0.1], [0.05, 0.85]])
K_mat_true = np.array([[0.6, 0.05], [0.05, 0.5]])
Omega_true = np.array([[1.0, 0.2], [0.2, 0.8]])
nu_true = 10.0

y_sim, mu_sim, _ = simulate_multivariate_t_gas(
    T=T,
    N=N,
    omega=omega_true,
    Phi=Phi_true,
    K_mat=K_mat_true,
    Omega=Omega_true,
    nu=nu_true,
    beta=None,  # No regressors explicitly
    seed=8888
)

print("✅ GAS simulation without regressors explicitly completed.")


100%|██████████| 500/500 [00:00<00:00, 29799.67it/s]

✅ GAS simulation without regressors explicitly completed.


In [43]:
# Explicit simulation parameters with regressors
T = 500
N = 2
K = 3
omega_true = np.array([0.5, -0.2])

Phi_true = np.array([[0.8, 0.1], [0.05, 0.85]])
K_mat_true = np.array([[0.6, 0.05], [0.05, 0.5]])
beta_true = np.array([[1.0, -0.5], [-1.2, 0.8], [0.5, 1.5]])
Omega_true = np.array([[1.0, 0.2], [0.2, 0.8]])
nu_true = 10.0

y_mtg, mu_mtg, X_mtg = simulate_multivariate_t_gas(
    T=T,
    N=N,
    omega=omega_true,
    Phi=Phi_true,
    K_mat=K_mat_true,
    beta=beta_true,  # With regressors explicitly
    Omega=Omega_true,
    nu=nu_true,
    # intercept=False,  # Explicitly include intercept
    seed=8888
)

print("✅ GAS simulation with regressors explicitly completed.")


100%|██████████| 500/500 [00:00<00:00, 20376.53it/s]

✅ GAS simulation with regressors explicitly completed.


In [44]:
# # Assuming you've run the simulation:
# # y_sim, mu_sim, X_sim = simulate_multivariate_gas(...)

# T, N = y_sim.shape
# time_axis = np.arange(T)

# fig = go.Figure()

# # Explicitly plot observations (y)
# for dim in range(N):
#     fig.add_trace(go.Scatter(
#         x=time_axis,
#         y=y_sim[:, dim],
#         mode='lines',
#         name=f'Observation y[{dim+1}]',
#         line=dict(width=1.5),
#         opacity=0.7
#     ))

# # Explicitly plot latent states (mu)
# for dim in range(N):
#     fig.add_trace(go.Scatter(
#         x=time_axis,
#         y=mu_sim[:, dim],
#         mode='lines',
#         name=f'Latent state μ[{dim+1}]',
#         line=dict(width=2, dash='dash')
#     ))

# # Explicit layout configuration
# fig.update_layout(
#     title="Multivariate GAS Simulated Data and Latent States",
#     xaxis_title="Time",
#     yaxis_title="Value",
#     legend=dict(x=1.05, y=1),
#     width=950,
#     height=600,
#     template="plotly_white"
# )

# fig.show()

time_axis = np.arange(T)
K = X_mtg.shape[1]

# Create subplots: N rows for dimensions + 1 for regressors
fig = make_subplots(
    rows=N + 1, cols=1, shared_xaxes=True,
    subplot_titles=[f"Dimension {i+1}" for i in range(N)] + ["Regressors X_t"]
)

# --- Add y and mu plots per dimension ---
for dim in range(N):
    fig.add_trace(go.Scatter(
        x=time_axis, y=y_mtg[:, dim],
        mode='lines', name=f'Observation y[{dim+1}]',
        line=dict(width=1.5), opacity=0.7
    ), row=dim+1, col=1)

    fig.add_trace(go.Scatter(
        x=time_axis, y=mu_mtg[:, dim],
        mode='lines', name=f'Latent state μ[{dim+1}]',
        line=dict(width=2, dash='dash')
    ), row=dim+1, col=1)

# --- Plot each regressor in final subplot row ---
for k in range(K):
    fig.add_trace(go.Scatter(
        x=time_axis, y=X_mtg[:, k],
        mode='lines', name=f'Regressor X[{k+1}]',
        line=dict(width=1)
    ), row=N+1, col=1)

# --- Layout styling ---
fig.update_layout(
    height=300 * (N + 1),
    width=950,
    title_text="Multivariate t-GAS Simulation: Observations, Latent States, and Regressors",
    template="plotly_white",
    showlegend=True
)

fig.update_xaxes(title_text="Time", row=N+1, col=1)
fig.update_yaxes(title_text="Value")

fig.show()



In [42]:
# # Explicitly create subplots
# fig = make_subplots(rows=N, cols=1, shared_xaxes=True,
#                     subplot_titles=[f"Dimension {i+1}" for i in range(N)])

# for dim in range(N):
#     # Plot observations explicitly
#     fig.add_trace(go.Scatter(
#         x=time_axis, y=y_sim[:, dim],
#         mode='lines', name=f'Observation y[{dim+1}]',
#         line=dict(width=1.5), opacity=0.7
#     ), row=dim+1, col=1)

#     # Plot latent states explicitly
#     fig.add_trace(go.Scatter(
#         x=time_axis, y=mu_sim[:, dim],
#         mode='lines', name=f'Latent state μ[{dim+1}]',
#         line=dict(width=2, dash='dash')
#     ), row=dim+1, col=1)

# fig.update_layout(
#     height=300*N, width=950,
#     title_text="Explicit Multivariate GAS Data: Observations vs. Latent States",
#     template="plotly_white"
# )

# fig.update_xaxes(title_text="Time", row=N, col=1)
# fig.update_yaxes(title_text="Value")

# fig.show()
